In [ ]:
# !pip install python-dotenv
# !pip install lyricsgenius

In [ ]:
import os

from dotenv import load_dotenv
from joblib import Parallel, delayed

load_dotenv()
API_TOKEN = os.getenv('TOKEN')

names = ['Gorillaz',
         'Muse', 'Metallica',
         'Flo rida', 'grandson',
         'Oomph!', 'Imagine dragons',
         'Michael jackson',
         'Placebo',
         'Blues saraceno',
         'System of a down',
         'Arctic monkeys',
         'Lmfao', 'Starset',
         'Oh wonder', 'Coldplay',
         'The prodigy', 'Disturbed',
         'The neighbourhood', 'Tardigrade inferno',
         'Scorpions', 'Shinoda', 'Fall out boy',
         'Dope', 'Keane', 'Linkin park',
         'Papa roach', 'The cranberries',
         'Neffex', 'Thirty seconds to mars',
         'Drake', 'Billie eilish', 'Queen', 'Ac dc',
         "Guns n roses", 'Eminem', 'Thousand foot krutch',
         'Meg myers', 'Led zeppelin', 'Skindred', 'Rammstein',
         'The pretty reckless', 'Nickelback', 'Bring me the horizon',
         'Halsey', 'Black eyed peas', 'Maroon 5', 'Hollywood undead', 'Nazareth']


def get_lyrics(name, k):
    import lyricsgenius as lg
    try:
        genius = lg.Genius(API_TOKEN, skip_non_songs=True,
                           excluded_terms=["(Remix)", "(Live)"],
                           remove_section_headers=True,
                           sleep_time=1,
                           retries=2,
                           verbose=True)
        songs = (genius.search_artist(name, max_songs=k, sort='popularity')).songs
        s = [song.lyrics for song in songs]
        with open(f"api/data/{name}.txt", "w") as f:
            f.write("\n \n".join(s))
        print(f"For {name} songs grabbed: {len(s)}")
    except:
        print(f"Some exception for {name}")


songs = Parallel(n_jobs=10, verbose=100)(delayed(get_lyrics)(i, 10) for i in names)

In [ ]:
import os
from dotenv import load_dotenv
import lyricsgenius as lg

load_dotenv()
API_TOKEN = os.getenv('TOKEN')


def name_checker(name):
    # if name not in names:
    try:
        genius = lg.Genius(API_TOKEN,
                           skip_non_songs=True,
                           remove_section_headers=True,
                           sleep_time=1,
                           retries=2,
                           verbose=True)
        response = (genius.search_artist(name, max_songs=1))
        true_name = response.name
        return true_name
    except:
        print(f"Some exception for {name}")
    # return name

    
for artist in os.listdir('api/data'):
    os.rename(f'api/data/{artist}',
              f'api/data/{name_checker(artist[:-4]).replace("/", "_")}.txt')

In [28]:
try:
    os.rename(f'api/data/\u200bgrandson.txt',
          f'api/data/Grandson.txt')
except:
    print("can't find artist named 'grandson'")

In [125]:
import os
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
import re
nltk.download('wordnet')
nltk.download('stopwords')
reg_tok = RegexpTokenizer('\w+')
stop_nltk = stopwords.words('english')
wn_lemmatizer = WordNetLemmatizer()


def clean_and_lemmatize(text, tokenizer=reg_tok, stopw=stop_nltk, lemmatizer=wn_lemmatizer):
    # cleaning
    text = text.lower()
    text = re.sub(r'http\S+', " ", text)
    text = re.sub(r'@\w+', ' ', text)
    text = re.sub(r'#\w+', ' ', text)
    text = re.sub(r'\d+', ' ', text)
    text = re.sub(r'<.*?>', ' ', text)
    
    # tokenining
    text = tokenizer.tokenize(text)

    # filtering from stopwords
#    text = [word for word in text if not word in stopw] # try with or without stopwords

    # lemmatization
    text = ' '.join([lemmatizer.lemmatize(word) for word in text])
    return text


songs_artists = dict()
for artist in os.listdir('api/data'):
    try:
        with open(f"api/data/{artist}", encoding='utf-8', newline='') as f:
            lyrics = f.read()
            songs_artists[artist[:-4]] = clean_and_lemmatize(lyrics)
    except:
        print(f'Some exception for {artist}')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\apofi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\apofi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


In [75]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

#from api.preprocessing import songs_artists

tfidf = TfidfVectorizer(ngram_range=(1, 4))
tfidf_representation = tfidf.fit_transform(songs_artists.values())
artists_similarity = cosine_similarity(tfidf_representation)

In [81]:
import pandas as pd
df = pd.DataFrame(artists_similarity, index=songs_artists.keys(), columns=songs_artists.keys())
df

,AC_DC,Arctic Monkeys,Billie Eilish,Black Eyed Peas,Blues Saraceno,Bring Me The Horizon,Coldplay,Disturbed,Drake,Eminem,...,Skindred,STARSET,System of a Down,Tardigrade Inferno,The Cranberries,The Neighbourhood,The Pretty Reckless,The Prodigy,Thirty Seconds to Mars,Thousand Foot Krutch
AC_DC,1.000000,0.105996,0.115232,0.090702,0.087866,0.122000,0.080099,0.122122,0.139701,0.192618,...,0.096118,0.138063,0.084968,0.091076,0.068773,0.121970,0.109239,0.051570,0.109377,0.105604
Arctic Monkeys,0.105996,1.000000,0.181856,0.120216,0.096950,0.165140,0.104237,0.172112,0.179979,0.244375,...,0.117253,0.178860,0.105618,0.123494,0.102372,0.168123,0.129017,0.058654,0.130345,0.155607
Billie Eilish,0.115232,0.181856,1.000000,0.129149,0.107769,0.210172,0.158947,0.193049,0.234758,0.291048,...,0.135979,0.209301,0.138280,0.137558,0.140444,0.233707,0.138860,0.060614,0.153218,0.187221
Black Eyed Peas,0.090702,0.120216,0.129149,1.000000,0.084143,0.136786,0.092181,0.148272,0.165206,0.217069,...,0.116047,0.144334,0.089961,0.102692,0.089873,0.132489,0.112752,0.101188,0.114247,0.155420
Blues Saraceno,0.087866,0.096950,0.107769,0.084143,1.000000,0.125404,0.074222,0.121815,0.125355,0.174739,...,0.087976,0.142716,0.082391,0.099278,0.070382,0.112050,0.113474,0.047757,0.106518,0.095943
Bring Me The Horizon,0.122000,0.165140,0.210172,0.136786,0.125404,1.000000,0.120431,0.194385,0.212808,0.279083,...,0.146453,0.222717,0.121677,0.146676,0.118145,0.200526,0.147805,0.064121,0.156352,0.184971
Coldplay,0.080099,0.104237,0.158947,0.092181,0.074222,0.120431,1.000000,0.117494,0.130465,0.299136,...,0.082527,0.132475,0.103664,0.088443,0.121716,0.140513,0.096044,0.034185,0.184673,0.098249
Disturbed,0.122122,0.172112,0.193049,0.148272,0.121815,0.194385,0.117494,1.000000,0.199501,0.281503,...,0.145211,0.225466,0.134618,0.150358,0.113185,0.182766,0.151560,0.072443,0.166540,0.178196
Drake,0.139701,0.179979,0.234758,0.165206,0.125355,0.212808,0.130465,0.199501,1.000000,0.318838,...,0.152309,0.218269,0.126787,0.149005,0.126302,0.226935,0.159840,0.068504,0.150062,0.191965
Eminem,0.192618,0.244375,0.291048,0.217069,0.174739,0.279083,0.299136,0.281503,0.318838,1.000000,...,0.204379,0.296450,0.181139,0.206223,0.171416,0.269053,0.221100,0.108074,0.243813,0.269148


In [120]:
print(f'You may like: {", ".join(df["Michael Jackson"].sort_values(ascending=False).index[1:6])}')


You may like: Eminem, Muse, Billie Eilish, Drake, NEFFEX


In [123]:
", ".join(df["Eminem"].sort_values(ascending=False).index[1:6])

'Hollywood Undead, Drake, NEFFEX, Led Zeppelin, Halsey'